In [1]:
def compare_strings_ignore_spaces(str1, str2):
    clean1 = str1.replace(" ", "")
    clean2 = str2.replace(" ", "")
    return clean1 == clean2

In [2]:
import pickle
def load_clients(pickle_path):
    try:
        with open(pickle_path, "rb") as f:
            clients = pickle.load(f)
        return clients
    except Exception as e:
        print(f"Failed to load pickle file: {e}")
        return None
full_data = load_clients("clients.pkl")

In [4]:
client_labels = []
for clients in full_data:
    if clients['label']['label'] == 'Reject':
        client_labels.append(False)
    elif clients['label']['label'] == 'Accept':
        client_labels.append(True)

In [ ]:
flags_preds = []       
for client in full_data:
    
    flags = True
    if client['passport']['first_name'] != client['account_form']['first_name']:
        flags = False
    if client['passport']['last_name'] != client['account_form']['last_name']:
        flags = False
    if client['passport']['middle_name'] != client['account_form']['middle_name']:
        flags = False
    if compare_strings_ignore_spaces((client['passport']['first_name']+client['passport']['middle_name']+client['passport']['last_name']), client['account_form']['name']) == False:
        flags = False
    if compare_strings_ignore_spaces((client['passport']['first_name']+client['passport']['middle_name']+client['passport']['last_name']), client['client_profile']['name']) == False:
        flags = False
    if client['passport']['gender'] != client['client_profile']['gender']:
        flags = False

    flags_preds.append(flags)

flags_preds = []       
for client in full_data:
    flags = True
    if client['passport']['passport_number'] != client['account_form']['passport_number']:
        flags = False
    flags_preds.append(flags)

from datetime import datetime

flags_preds = [] 
for client in full_data:
    flags = True
    expiry_str = client['passport']['passport_expiry_date']
    
    # Convert string to datetime object (adjust format if needed)
    expiry_date = datetime.strptime(client['passport']['passport_expiry_date'], "%Y-%m-%d").date()
    today = datetime.strptime("2021-04-10", "%Y-%m-%d").date() #2025-04-1  ?
    if expiry_date < today:
        flags = False

    
    flags_preds.append(flags)

from email_validator import validate_email, EmailNotValidError

def could_be_valid_email(email):
    try:
        validate_email(email, check_deliverability=False)
        return True
    except EmailNotValidError:
        return False
    
flags_preds = []       
for client in full_data:
    flags = True
    if could_be_valid_email(client['client_profile']['email_address']) == False:
        flags = False
    if client['account_form']['email_address'] != client['client_profile']['email_address']:
        flags = False   
    if client['account_form']['phone_number'] != client['client_profile']['phone_number']:
        flags = False 
    flags_preds.append(flags)

flags_preds = []       
for client in full_data:
    flags = True
    passport_birth_date = datetime.strptime(client['passport']['birth_date'], "%Y-%m-%d").date()
    client_birth_date = datetime.strptime(client['client_profile']['birth_date'], "%Y-%m-%d").date()
    passport_issued_date = datetime.strptime(client['passport']['passport_issue_date'], "%Y-%m-%d").date()
    client_issued_date = datetime.strptime(client['client_profile']['passport_issue_date'], "%Y-%m-%d").date()
    passport_expiry_date = datetime.strptime(client['passport']['passport_expiry_date'], "%Y-%m-%d").date()
    client_expiry_date = datetime.strptime(client['client_profile']['passport_expiry_date'], "%Y-%m-%d").date()
    if passport_expiry_date < passport_issued_date:
        flags = False
    if passport_birth_date != client_birth_date:
        flags = False
    if passport_issued_date != client_issued_date:
        flags = False
    if passport_expiry_date != client_expiry_date:
        flags = False
    if passport_issued_date < passport_birth_date:
        flags = False
    
    flags_preds.append(flags)

In [18]:
from sklearn.metrics import confusion_matrix
import numpy as np
print(confusion_matrix(np.array(flags_preds), np.array(client_labels)))

[[  88    0]
 [4904 5008]]
